<a href="https://colab.research.google.com/github/leartssy/BlenderDiffuseNST/blob/main/DiffuseST_03_Tileability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer by DiffuseST #

# Initial Setup

Downgrade to python 3.11 as that is what Blender uses

In [2]:
!sudo apt-get update
!sudo apt-get install python3.11 python3.11-venv
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!python3 --version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [60.9 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,290 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubuntu

In [3]:
!python --version

Python 3.11.13


Clone my own github repo

In [21]:
!git clone https://github.com/leartssy/BlenderDiffuseNST.git

Cloning into 'BlenderDiffuseNST'...
remote: Enumerating objects: 4763, done.
remote: Counting objects: 100% (4763/4763), done.
remote: Compressing objects: 100% (4761/4761), done.
remote: Total 4763 (delta 20), reused 4732 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (4763/4763), 342.86 MiB | 22.95 MiB/s, done.
Resolving deltas: 100% (20/20), done.


Setup libraries and dependencies

In [2]:
!pip uninstall -y diffusers transformers accelerate peft huggingface-hub tokenizers

Found existing installation: diffusers 0.34.0
Uninstalling diffusers-0.34.0:
  Successfully uninstalled diffusers-0.34.0
Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Found existing installation: huggingface-hub 0.33.4
Uninstalling huggingface-hub-0.33.4:
  Successfully uninstalled huggingface-hub-0.33.4
Found existing installation: tokenizers 0.21.2
Uninstalling tokenizers-0.21.2:
  Successfully uninstalled tokenizers-0.21.2


In [5]:
%cd BlenderDiffuseNST/DiffuseST_with_TexTile/
!pip install -r requirements.txt
%cd ..

/content/BlenderDiffuseNST/DiffuseST_with_TexTile
/content/BlenderDiffuseNST


Restart Runtime

In [6]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")

CUDA is available! Device name: Tesla T4


Load the blipdiffusion model

In [7]:
!git clone https://huggingface.co/salesforce/blipdiffusion

Cloning into 'blipdiffusion'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 27 (from 1)
Unpacking objects: 100% (32/32), 520.37 KiB | 2.23 MiB/s, done.
Filtering content: 100% (6/6), 8.24 GiB | 173.96 MiB/s, done.


In [27]:
%cd blipdiffusion/

[Errno 2] No such file or directory: 'blipdiffusion/'
/content/BlenderDiffuseNST/DiffuseST_with_TexTile/BlenderDiffuseNST/DiffuseST_with_TexTile


In [28]:
%cd /content/BlenderDiffuseNST/blipdiffusion

/content/BlenderDiffuseNST/blipdiffusion


In [30]:
# The 'blipdiffusion' repository was cloned to:
cloned_model_key = !pwd
print("The model was cloned to {cloned_model_key}")

/content/BlenderDiffuseNST/blipdiffusion


Convert saved model.bin files to safetensors - because of version compatibility

In [9]:
from safetensors.torch import save_file
import torch
%cd "unet"
# Load the PyTorch .bin model
state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors
save_file(state_dict, "diffusion_pytorch_model.safetensors")

/content/BlenderDiffuseNST/blipdiffusion/unet


In [10]:
%cd ..
%cd "vae"
# Load the PyTorch .bin model
state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors in the same folder
save_file(state_dict, "diffusion_pytorch_model.safetensors")

/content/BlenderDiffuseNST/blipdiffusion
/content/BlenderDiffuseNST/blipdiffusion/vae


In [11]:
%cd ..
%cd ..

/content/BlenderDiffuseNST/blipdiffusion
/content/BlenderDiffuseNST


In [22]:
%cd DiffuseST_with_TexTile/

[Errno 2] No such file or directory: 'DiffuseST_with_TexTile/'
/content/BlenderDiffuseNST/DiffuseST_with_TexTile


Install TexTile

In [23]:
!pip install textile-metric

In [24]:
#initialize the textile loss
import textile
from textile.utils.image_utils import read_and_process_image

loss_textile = textile.Textile()

#Run normal Diffusion Style Transfer

In [25]:
%cd BlenderDiffuseNST/DiffuseST_with_TexTile

/content/BlenderDiffuseNST/DiffuseST_with_TexTile/BlenderDiffuseNST/DiffuseST_with_TexTile


Choose the content and style image directory as well as the output path

In [19]:
content_path = str(input("Enter the content image path: "))
style_path = str(input("Enter the style image"))
output_path = str(input("Enter the output path: "))

Enter the content image path: "C:\Users\leaes\Documents\Studium\7. Semester\Bachelor\Bachelor Thesis\Example Pictures\TestDiffuse\content"
Enter the style image"C:\Users\leaes\Documents\Studium\7. Semester\Bachelor\Bachelor Thesis\Example Pictures\TestDiffuse\style"
Enter the output path: "C:\Users\leaes\Documents\Studium\7. Semester\Bachelor\Bachelor Thesis\Example Pictures\TestDiffuse\output"


In [10]:
#display all option for run.py
!python run.py --help

2025-11-26 15:54:35.034129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764172475.054139    6808 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764172475.060184    6808 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: run.py [-h] [--content_path CONTENT_PATH] [--style_path STYLE_PATH]
              [--output_dir OUTPUT_DIR] [--sd_version {1.5,2.0,2.1}]
              [--device DEVICE] [--alpha ALPHA] [--seed SEED]
              [--ddpm_steps DDPM_STEPS] [--steps_to_save STEPS_TO_SAVE]
              [--ddim_steps DDIM_STEPS]
              [--textile_guidance_scale TEXTILE_GUIDANCE_SCALE]
              [--inversion_prompt INVERSION_PROMPT] [--

Run diffusion style transfer: put content images in images/content and style images in images/style

In [26]:
#diffusion style transfer without tileability = texture_guidance_scale 0.0
!python run.py --content_path {content_path} --style_path {style_path} --output_dir {output_path} --textile_guidance_scale 0.0

2025-11-26 16:37:25.253605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764175045.277976    3973 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764175045.288238    3973 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/content/BlenderDiffuseNST/DiffuseST_with_TexTile/BlenderDiffuseNST/DiffuseST_with_TexTile/run.py", line 208, in <module>
    run(opt)
  File "/content/BlenderDiffuseNST/DiffuseST_with_TexTile/BlenderDiffuseNST/DiffuseST_with_TexTile/run.py", line 27, in run
    blip_diffusion_pipe = BLIP.from_pretrained(model_key, torch_dtype=torch.float16).to("cuda")
                          ^^^^^^^^^^^^^

# Adding Tileability by TexTile #

Textile_guidance_scale values:
1.0-10.0: low
10.0-20.0: medium
20.0+:high

In [ ]:
!python run.py --textile_guidance_scale 25.0 --alpha 0.4 --ddim_steps 50

2025-11-26 13:05:44.244268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764162344.264026    2556 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764162344.270097    2556 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading pipeline components...:  86% 6/7 [01:20<00:11, 11.44s/it]/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 442kB/s]

vocab.txt: 100% 232k/232k